In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re

In [8]:
def get_num_items(x, value_to_find:str):
    idx = [index+1 for index, value in enumerate(x) if value == value_to_find]
    return str(dict(Counter(np.array(x)[idx])))


reviews = pd.read_csv('./reviews_total/reviews_total.csv', delimiter=';')
reviews = reviews.dropna(subset=['language'])
# reviews.expertai_entities.apply(lambda x: np.delete(re.sub('[\]\"  \[\,]','', x).split("'"),''))
reviews['entities'] = reviews.expertai_entities.apply(lambda x: [i for i in re.sub('[\]\"  \[\,]','', x).split("'") if i!=''])
reviews['items'] = reviews['expertai_sentiment.items'].apply(lambda x: [i for i in re.sub('[\]\"  \[\,]','', x).split("'") if i!=''])
reviews['phrases'] = reviews['expertai_mainPhrases'].apply(lambda x: [i for i in re.sub('[\]\"\[\,]','', x).split("'") if i!=''])
reviews['lemmas'] = reviews['expertai_mainLemmas'].apply(lambda x: [i for i in re.sub('[\]\"\[\,]','', x).split("'") if i!=''])
reviews['isAgent'] = reviews.place_name.apply(lambda x: True if 'Agent' in x else False)

In [9]:
reviews['match'] = reviews.apply(lambda x: 1 if (x.rating >= 4 and x['sentiment.overall']>0) or (x.rating < 4 and x['sentiment.overall']<0) else 0, axis=1)

In [28]:
mismatches = reviews[reviews.match == 0][[
    'rating', 'sentiment.overall', 'text', 'items', 'lemmas', 'phrases'
]].sort_values(['rating', 'sentiment.overall'], ascending=[True, False]).drop_duplicates(subset=['text'])#.to_excel('./data/mismatch.xlsx')

In [30]:
len(mismatches) / len(reviews)

0.14630380235475776

In [29]:
mismatches.head(10)

,rating,sentiment.overall,text,items,lemmas,phrases
2022,1,51.20,I don't use this insurance company,"[insurancecompany, 3.5]","[insurance company, 100.0]",[]
7963,1,31.10,Don’t waste your time or money.,"[money, 8.5, time, 8.5]","[money, 100.0]",[]
2432,1,29.60,Stil they didn’t give my friends and a lot of ...,"[-3.5, company, 12.0]","[money, 39.29 , people, 20.29 , company, 20...","[a lot of peoples money, 93.4 , peoples money..."
7583,1,28.79,The phone run around just trying to talk to so...,"[14.0, phone, 3.5]","[phone, 55.7 , attempt, 44.2]",[]
5890,1,25.60,Still trying to get resolve from roofing damag...,[14.0],"[damage, 58.9 , roofing, 41.0]","[roofing damage, 99.0 , resolve from roofing ..."
650,1,24.70,Rates very hi.,[4.5],"[rate, 53.2 , hi, 46.7]",[]
3895,1,22.60,I'm going to keep this short and sweet: stay a...,[6.5],"[guy, 70.9 , short and sweet, 29.0]",[]
5674,1,20.00,Customer service very nice to me but that ACCI...,"[2.4, 800dollar, -3.5, customerservice, 6.4, j...","[forgiveness, 27.2 , accident, 19.1 , joke, ...","[accident forgiveness, 95.3 , have a accident..."
5895,1,19.70,Got my friend another insurance company much b...,"[6.0, price, 3.0]","[insurance company, 47.9 , price, 30.5 , fri...","[better price, 100.0]"
30,1,19.20,Sucks.no stars,"[Sucks.no, 3.5]","[Sucks.no, 100.0]",[]


In [31]:
mismatches.tail(10)

,rating,sentiment.overall,text,items,lemmas,phrases
1482,5,-23.70,Very Professional and educational! Got me the ...,[-9.39],"[thanks, 45.29 , corner, 27.1 , coverage, 2...","[corners m thanks, 93.09 , m thanks, 6.8]"
1275,5,-25.20,I was in a bind and needed insurance on my hom...,"[-5.0, Lisa, -2.5, type, -3.5]","[Lisa, 35.0 , customer service, 27.29 , insu...","[type of customer service, 68.19 , needed ins..."
2682,5,-26.39,Got a cheap quote.,"[quote, -3.0]","[quote, 97.4 , cheap, 2.5]","[cheap quote, 98.5 , get a cheap quote, 1.39]"
3375,5,-28.60,Won't find a better team of Insurance Profesio...,"[InsuranceProfesionals, -3.0, team, -3.5]","[Insurance Profesionals, 37.9 , team, 36.5 ,...","[team of Insurance Profesionals, 71.0 , bette..."
3773,5,-30.79,So far no complaints thank you,"[-3.5, complaint, -3.5]","[complaint, 92.69 , no, 7.19]",[]
4896,5,-35.20,Rob s team is great !,"[team, -6.0]","[team, 97.59 , great, 2.29]","[s team, 98.19 , rob s team, 1.7]"
3235,5,-52.79,Hassle free.,[-6.0],"[hassle, 98.9 , free, 1.0]",[]
5557,5,-57.59,Ben got us a great deal on our car and home in...,[-7.0],"[Ben, 25.7 , great deal, 19.79 , insurance, ...","[home insurance, 90.4 , have question, 9.5]"
651,5,-61.59,Liberty liberty liberty liberty,[-3.5],"[liberty, 100.0]","[liberty liberty liberty liberty, 92.4 , libe..."
1387,5,-88.00,Wrong place.,"[place, -5.0]","[place, 100.0]","[wrong place, 100.0]"
